In [ ]:
git clone https://github.com/Pyran1/MalwareDatabase
# this is a collection of malware that we will use as an example of a dataset

# Organizing subfolders into folders , assuming "MalwareDatabase" is your malwares folder


In [ ]:
source_path = r'./MalwareDatabase'
destination_path = r'./organized_malwareCONCAT'
for item in os.listdir(source_path):
        item_path = os.path.join(source_path, item)
        if os.path.isdir(item_path) and ".git" not in item_path:
          root,subdir,files = list(os.walk(item_path))[0]
          print(root)
          if len(subdir) != 0:
            dest_dir = os.path.join(destination_path, os.path.basename(root))
            os.makedirs(dest_dir, exist_ok=True)
            for sd in subdir:
              subfolder_path=os.path.join(root,sd)
              for file in os.listdir(subfolder_path):
                source_file = os.path.join(subfolder_path, file)
                dest_file = os.path.join(dest_dir, file)
                shutil.copy(source_file, dest_file)
                print(f"Moved {source_file} to {dest_file}")
          else:
            dest_dir = os.path.join(destination_path, os.path.basename(root))
            os.makedirs(dest_dir, exist_ok=True)
            for file in os.listdir(root):
              source_file = os.path.join(root, file)
              dest_file = os.path.join(dest_dir, file)
              shutil.copy(source_file, dest_file)
              print(f"Moved {source_file} to {dest_file}")


# unzipping the compressed files 


In [ ]:
import os
import subprocess

organized_malware_path = r'./organized_malwareCONCAT'
unzip_destination = r'./unzipped_malwares'
password = "infected"  # Password for encrypted files

for root, dirs, files in os.walk(organized_malware_path):
    for file in files:
        file_path = os.path.join(root, file)

        if file.endswith(('.zip', '.7z', '.rar')): 
            dest_dir = os.path.join(unzip_destination, os.path.relpath(root, organized_malware_path))
            os.makedirs(dest_dir, exist_ok=True)
            
            # Unzip the file using 7-Zip
            try:
                subprocess.run(
                    [r"C:\Program Files\7-Zip\7z.exe", "x", file_path, f"-o{dest_dir}", f"-p{password}"],
                    check=True
                )
                print(f"Unzipped {file_path} to {dest_dir}")
            except subprocess.CalledProcessError as e:
                print(f"Failed to unzip {file_path} with 7-Zip: {e}")


# check that no folder contains a subfolder

In [ ]:

import os
import shutil  


source_path = r'./unzipped_malwares'


for item in os.listdir(source_path):
  item_path = os.path.join(source_path, item)
  if os.path.isdir(item_path):
    for root, dirs, files in os.walk(item_path):
      if dirs:  # Check if the 'dirs' list is not empty
        for i in range(len(dirs)):
          print(f"Folder '{item}' contains subfolders: {dirs[i]}")
          shutil.rmtree(os.path.join(item_path,dirs[i]))


# converting the binary files into images 


In [ ]:
import os
import numpy as np
from PIL import Image

def hex2img(array, output_img_path):
    if array.shape[1] != 16:  
        raise ValueError(f"Expected array shape to have 2nd dimension of size 16, but got {array.shape[1]}")

    b = int((array.shape[0] * 16) ** (0.5))
    b = 2 ** (int(np.log2(b)) + 1)
    a = int(array.shape[0] * 16 / b)
    print(a, b, array.shape)

    array = array[:a * b // 16, :]
    array = np.reshape(array, (a, b))

    im = Image.fromarray(np.uint8(array))

    im = im.resize((224, 224), Image.LANCZOS)

    im.save(output_img_path, "PNG")
    return im

unzipped_malwares_path = r'unzipped_malwares'
malware_image_path = r'malware_Image'

os.makedirs(malware_image_path, exist_ok=True)

binary_extensions = [
    '.exe', '.dll', '.macho', '.hta', '.wsf', '.apk', '.msi', '.bat', '.cmd',
    '.xlam', '.arj', '.r00', '.ppam', '.7z', '.zip', '.html', '.xls', '.xz',
    '.r07', '.gz', '.sh', '.pdf', '.lnk', '.a', '.docx', '.jar', '.com', '.z',
    '.php', '.img', '.r09', '.iso', '.unknown', '.js', '.xlsm', '.rtf', '.elf',
    '.xlsx', '.bin', '.rar', '.docm', '.ps1', '.vbs', '.doc', '.class'
]

for root, dirs, files in os.walk(unzipped_malwares_path):
    for name in files:
        if any(name.endswith(ext) for ext in binary_extensions):  # Check against the new list of extensions
            file_path = os.path.join(root, name)
            relative_path = os.path.relpath(root, unzipped_malwares_path)
            output_folder = os.path.join(malware_image_path, relative_path)
            os.makedirs(output_folder, exist_ok=True)

            print(f'Processing binary file: {file_path}')
            with open(file_path, 'rb') as f:
                file_data = bytearray(f.read())

            hex_array = []
            for count, byte in enumerate(file_data):
                hex_value = f'{byte:02x}' 
                hex_array.append(hex_value)

            array = []
            for i in range(0, len(hex_array), 16):
                if i + 16 <= len(hex_array):  # Ensure there are enough elements
                    row = [int(hex_array[j], 16) for j in range(i, i + 16)]
                    array.append(row)

            if not array:
                print(f"No valid hex data for file: {file_path}")
                continue

            array = np.array(array)

            output_image_path = os.path.join(output_folder, name.split('.')[0] + ".png")
            hex2img(array, output_image_path)
            print(f"Image saved at: {output_image_path}")


In [ ]:
import os
import shutil

def copy_folders_with_min_files(data_dir, min_files, destination_dir):
    """
    Traverses the subfolders of the given directory, checks if they contain
    at least `min_files` files, and copies those subfolders to a destination directory.

    Parameters:
    - data_dir: str, the path of the directory to traverse.
    - min_files: int, the minimum number of files a subfolder should have to be copied.
    - destination_dir: str, the path to the destination folder where qualified subfolders will be copied.
    """
    os.makedirs(destination_dir, exist_ok=True)

    for root, dirs, files in os.walk(data_dir):
        num_files = len(files)  
        if num_files >= min_files:
            relative_path = os.path.relpath(root, data_dir)
            destination_path = os.path.join(destination_dir, relative_path)

            shutil.copytree(root, destination_path)

            print(f"Copied folder: {root} - {num_files} files to {destination_path}")

data_dir = r'malware_Image' 
min_files = 20 
destination_dir = r'malware_Image_richer_class'  

copy_folders_with_min_files(data_dir, min_files, destination_dir)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

# Path to the image to be predicted
image_path = r'malware_Image\7z\40f4034a2b5d7721785bcae61fe4d8ca5986ac44955f632bc1585c8df39509bc.png'

# Load the model (with the correct path to the saved model)
model = load_model('malware_model_checkpoints.h5.keras')

# Load the class labels from the previous training (using the `train_gen.class_indices`)
class_names = list(train_gen.class_indices.keys())

# Load and preprocess the image
img = image.load_img(image_path, target_size=(256, 256))  # Resize to the target size
img_array = image.img_to_array(img)  # Convert image to numpy array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the image to [0, 1] range

# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Display the image and the prediction
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f'Predicted Class: {class_names[predicted_class[0]]}')
plt.axis('off')  # Hide axes for better visualization
plt.show()

# Print the predicted class
print(f'Predicted class: {class_names[predicted_class[0]]}')


#Note to Self: Tomorrow load MyDrive/TSYP_2024/organized_malwareCONCAT

##pass the unzipper on it.the subfold_remover on it then the image converter on it
#do it for atleast 4 folders so that it's doable since the whole data is 7 hours of processing

#then pass it on the train code!